In [ ]:
#Import Libraries
import pandas as pd
import numpy as np
from numpy import percentile
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import re # for regular expressions
import pandas as pd 
pd.set_option("display.max_colwidth", 200) 
import string
import nltk # for text manipulation
from nltk.stem.porter import *
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from tqdm import tqdm
from gensim.models.doc2vec import LabeledSentence
import gensim
from sklearn.linear_model import LogisticRegression
from scipy import stats 
from sklearn import metrics 
from sklearn.metrics import mean_squared_error,mean_absolute_error, make_scorer,classification_report,confusion_matrix,accuracy_score,roc_auc_score,roc_curve
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.naive_bayes import BernoulliNB
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import xgboost as xgb
import warnings 
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
#read data from file
df=pd.read_csv('https://raw.githubusercontent.com/vishallanka2089/sentiment-analysis-of-tweets/main/Corona_NLP%20(2)%20(1).csv',encoding='latin1')

In [ ]:
#Reviewing our data set
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate su...,Positive
2,3801,48753,Vagabonds,16-03-2020,"Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P",Positive
3,3802,48754,NaN,16-03-2020,"My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\...",Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COVID19 outbreak.\r\r\n\r\r\nNot because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thing, but please, don...",Extremely Negative


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       41157 non-null  int64 
 1   ScreenName     41157 non-null  int64 
 2   Location       32567 non-null  object
 3   TweetAt        41157 non-null  object
 4   OriginalTweet  41157 non-null  object
 5   Sentiment      41157 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.9+ MB


In [ ]:
df.shape

(41157, 6)

In [ ]:
df.columns

Index(['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet',
       'Sentiment'],
      dtype='object')

In [ ]:
df.isnull().sum()

UserName            0
ScreenName          0
Location         8590
TweetAt             0
OriginalTweet       0
Sentiment           0
dtype: int64

In [ ]:
#Converting extremely positive and extremely negative to positive and negative respectively
df['Sentiment'].value_counts()

Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

In [ ]:
df['Sentiment']=df['Sentiment'].replace({'Extremely Positive': 'Postitive', 'Extremely Negative': 'Negative'})
df['Sentiment'].value_counts()

Negative     15398
Positive     11422
Neutral       7713
Postitive     6624
Name: Sentiment, dtype: int64

In [ ]:
df['OriginalTweet'].sample(2)


6266     @charcuterie10 @adamseconomics That is correct. This will be a source of further financial destruction within the #USA economy, as overseas income of multinationals, based in USA, lose value as th...
35568    #ScamAlert: Phishing emails claiming to be from BBB are making the rounds - offering #COVID19 related products like antiviral hand sanitizer. Don't respond and don't click links, and always check ...
Name: OriginalTweet, dtype: object

In [ ]:
df['OriginalTweet'][1823]


"Saks Fifth Avenue has closed their Manhattan Flagship store for up to 2 weeks! Watch this livestream showing Saks Store, Fifth Avenue, St Patrick's Cathedral to see how empty NYC is, #StPatricksDay Parade would have taken place here #retail #CoronaVirus   https://t.co/GnV8Jbewab"

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
 import nltk
 nltk.download('stopwords')
 nltk.download('punkt')
 nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
lb=WordNetLemmatizer()
sw=stopwords.words('english')
print(sw)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
corpus=[]
for i in df['OriginalTweet']:
    txt=re.sub('[^A-Za-z0-9]',' ',i)         #Removing punctuations
    txt=txt.lower()                          #Converting to Lower case
    txt=word_tokenize(txt)                    #Tokenizing the words
    txt=[i for i in txt if i not in sw]       #Removing stop words
    txt=[lb.lemmatize(i) for i in txt]         #Lemmatization                                                  
    txt= " ".join(txt)
    corpus.append(txt)

In [ ]:
print(corpus[:10])

['menyrbie phil gahan chrisitv http co ifz9fan2pa http co xx6ghgfzcc http co i2nlzdxno8', 'advice talk neighbour family exchange phone number create contact list phone number neighbour school employer chemist gp set online shopping account po adequate supply regular med order', 'coronavirus australia woolworth give elderly disabled dedicated shopping hour amid covid 19 outbreak http co binca9vp8p', 'food stock one empty please panic enough food everyone take need stay calm stay safe covid19france covid 19 covid19 coronavirus confinement confinementotal confinementgeneral http co zrlg0z520j', 'ready go supermarket covid19 outbreak paranoid food stock litteraly empty coronavirus serious thing please panic cause shortage coronavirusfrance restezchezvous stayathome confinement http co usmualq72n', 'news region first confirmed covid 19 case came sullivan county last week people flocked area store purchase cleaning supply hand sanitizer food toilet paper good tim dodson report http co cfxch7

In [ ]:
#vectorizer
tf=TfidfVectorizer(max_features=5000)
tf_res=tf.fit_transform(corpus).toarray() 

In [ ]:
print(tf_res[:5])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
print(type(tf_res))

<class 'numpy.ndarray'>


In [ ]:
x=tf_res
y=df['Sentiment']
print(len(x))
print (y.shape)

41157
(41157,)


In [ ]:
x_train,x_test,y_train, y_test=train_test_split(x,y,test_size=0.5)
print(x_train. shape)
print(x_test.shape)
print (y_train. shape)
print(y_test.shape)

(20578, 5000)
(20579, 5000)
(20578,)
(20579,)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#Multinomial Naive Bayes Classification
m1=MultinomialNB()
m1.fit(x_train,y_train)

MultinomialNB()

In [ ]:
print('Training Score',m1.score(x_train,y_train))
print('Testing Score',m1.score(x_test,y_test))

Training Score 0.652006997764603
Testing Score 0.5454103697944507


In [ ]:
ypred_m1=m1.predict(x_test)
print (ypred_m1)

['Postitive' 'Positive' 'Negative' ... 'Positive' 'Positive' 'Negative']


In [ ]:
cm_m1=confusion_matrix(y_test,ypred_m1)
print(cm_m1)
print(classification_report(y_test,ypred_m1))

[[6760  182  696   36]
 [1915 1068  839   33]
 [2569  276 2733  178]
 [ 646   59 1926  663]]
              precision    recall  f1-score   support

    Negative       0.57      0.88      0.69      7674
     Neutral       0.67      0.28      0.39      3855
    Positive       0.44      0.47      0.46      5756
   Postitive       0.73      0.20      0.32      3294

    accuracy                           0.55     20579
   macro avg       0.60      0.46      0.46     20579
weighted avg       0.58      0.55      0.51     20579



In [ ]:
# RANDOM FOREST CLASSIFIER    (THIS BLOCK OF CODE TAKES ABOUT 2 MINUTES TO EXECUTE)
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier()
rf_clf.fit(x_train,y_train)
rf_prediction = rf_clf.predict(x_test)
cm_m2=confusion_matrix(y_test,rf_prediction)
print(cm_m2)
print(classification_report(rf_prediction,y_test))

[[6142  799  589  144]
 [1259 2197  356   43]
 [2235  852 2169  500]
 [ 611  155 1142 1386]]
              precision    recall  f1-score   support

    Negative       0.80      0.60      0.69     10247
     Neutral       0.57      0.55      0.56      4003
    Positive       0.38      0.51      0.43      4256
   Postitive       0.42      0.67      0.52      2073

    accuracy                           0.58     20579
   macro avg       0.54      0.58      0.55     20579
weighted avg       0.63      0.58      0.59     20579



In [ ]:
#KNN CLASSIFIER                     (THIS BLOCK OF CODE TAKES ABOUT 2 MINUTES TO EXECUTE)
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3,leaf_size=25)
knn.fit(x_train, y_train)
y_pred_knn=knn.predict(x_test)
print(classification_report(y_test,y_pred_knn))
cm_rfc=confusion_matrix(y_test, y_pred_knn)
print(cm_rfc)

              precision    recall  f1-score   support

    Negative       0.55      0.46      0.50      7674
     Neutral       0.24      0.75      0.37      3855
    Positive       0.44      0.13      0.20      5756
   Postitive       0.60      0.11      0.18      3294

    accuracy                           0.37     20579
   macro avg       0.46      0.36      0.31     20579
weighted avg       0.47      0.37      0.34     20579

[[3561 3729  333   51]
 [ 713 2901  204   37]
 [1486 3379  746  145]
 [ 686 1835  426  347]]


Hence it is clear that Random forest classifier has best accuracy